# General Usage

In [1]:
from pprint import pprint
import tdameritrade as td
import os
import json

## Authentication



In [2]:
# get refreshtoken from file
refreshtoken = open(os.path.expanduser('~/.r_token'), 'r')
line = refreshtoken.readline()
refreshtoken.close()

client_id = os.getenv('TDAMERITRADE_CLIENT_ID')
account_id = os.getenv('TDAMERITRADE_ACCOUNT_ID')

# authenticate and initialize TD Client
td_client = td.TDClient(client_id=client_id, refresh_token=line)



## API Examples

In [3]:
# shows private account info
#pprint(td_client.accounts())

### Getting Symbol Data

In [4]:
pprint(td_client.search('aapl'))

{'AAPL': {'assetType': 'EQUITY',
          'cusip': '037833100',
          'description': 'Apple Inc. - Common Stock',
          'exchange': 'NASDAQ',
          'symbol': 'AAPL'}}


In [30]:
pprint(td_client.quote('gsx'))

{'GSX': {'52WkHigh': 149.05,
         '52WkLow': 27.06,
         'askId': 'N',
         'askPrice': 92.8,
         'askSize': 200,
         'assetMainType': 'EQUITY',
         'assetSubType': 'ADR',
         'assetType': 'EQUITY',
         'bidId': 'T',
         'bidPrice': 92.67,
         'bidSize': 200,
         'bidTick': ' ',
         'closePrice': 91.33,
         'cusip': '36257Y109',
         'delayed': False,
         'description': 'GSX Techedu Inc. American Depositary Shares, three of '
                        'which representing two Cla',
         'digits': 2,
         'divAmount': 0.0,
         'divDate': '',
         'divYield': 0.0,
         'exchange': 'n',
         'exchangeName': 'NYSE',
         'highPrice': 93.98,
         'lastId': 'V',
         'lastPrice': 92.735,
         'lastSize': 100,
         'lowPrice': 88.65,
         'marginable': True,
         'mark': 92.735,
         'markChangeInDouble': 1.405,
         'markPercentChangeInDouble': 1.5384,
         'nA

In [29]:
pprint(td_client.instrument('037833100'))

[{'assetType': 'EQUITY',
  'cusip': '037833100',
  'description': 'Apple Inc. - Common Stock',
  'exchange': 'NASDAQ',
  'symbol': 'AAPL'}]


In [7]:
import pandas as pd
pd.DataFrame(td_client.history('AAPL')['candles']).head()

,open,high,low,close,volume,datetime
0,143.37,144.50,143.27,144.12,73774,1611748800000
1,144.06,144.36,144.06,144.30,31926,1611748860000
2,144.29,144.50,144.21,144.50,17191,1611748920000
3,144.50,144.52,144.20,144.29,16861,1611748980000
4,144.25,144.40,144.23,144.31,18081,1611749040000


### Getting Option Data

In [8]:
options = td_client.options('GSX')

In [31]:
options = td_client.options('GSX')
expiry_dates = list(options['callExpDateMap'].keys())
#expiry = expiry_dates[0]
expiry = '2021-02-19:9'
#print(f"Expiry Date: {expiry}")
#pprint(expiry_dates)

strike_prices = list(options['callExpDateMap'][expiry].keys())
#strike = strike_prices[0]
strike = '70.0'
#print(f"Strike Price: {strike}")
#pprint(strike_prices)

pprint(options['callExpDateMap'][expiry][strike])

[{'ask': 24.05,
  'askSize': 12,
  'bid': 23.1,
  'bidAskSize': '2X12',
  'bidSize': 2,
  'closePrice': 22.44,
  'daysToExpiration': 9,
  'deliverableNote': '',
  'delta': 0.921,
  'description': 'GSX Feb 19 2021 70 Call',
  'exchangeName': 'OPR',
  'expirationDate': 1613768400000,
  'expirationType': 'R',
  'gamma': 0.007,
  'highPrice': 24.3,
  'inTheMoney': True,
  'isIndexOption': None,
  'last': 23.46,
  'lastSize': 0,
  'lastTradingDay': 1613782800000,
  'lowPrice': 23.0,
  'mark': 23.58,
  'markChange': 1.13,
  'markPercentChange': 5.04,
  'mini': False,
  'multiplier': 100.0,
  'netChange': 1.02,
  'nonStandard': False,
  'openInterest': 10990,
  'openPrice': 0.0,
  'optionDeliverablesList': None,
  'percentChange': 4.52,
  'putCall': 'CALL',
  'quoteTimeInLong': 1612974539576,
  'rho': 0.016,
  'settlementType': ' ',
  'strikePrice': 70.0,
  'symbol': 'GSX_021921C70',
  'theoreticalOptionValue': 23.175,
  'theoreticalVolatility': 29.0,
  'theta': -0.123,
  'timeValue': 0.79,
 

## Getting Account Positions

In [11]:
# PRIVATE account info
# pprint(td_client.accounts(positions=True))

## Placing an Order

### Creating order body for Place Order API

In [14]:
from tdameritrade.orders.order_builder import build_buy_market_stock_order

# Prints function docstring
# build_buy_market_stock_order?

In [15]:
symbol = "HUSA"  # Cheap stock for example
quantity = 1
order = build_buy_market_stock_order(symbol, quantity)

In [16]:
pprint(order.asdict())

{'duration': 'DAY',
 'orderLegCollection': [{'instruction': 'BUY',
                         'instrument': {'assetType': 'EQUITY',
                                        'symbol': 'HUSA'},
                         'quantity': 1}],
 'orderStrategyType': 'SINGLE',
 'orderType': 'MARKET',
 'session': 'NORMAL'}


### Submitting Order

In [18]:
# Without this, it fails with a 415.
# This may screw up other API calls, so it may have to be reset if you want to try other ones.
td_client.session.headers = {
    "Content-Type": "application/json"
}

In [19]:
# This will fail to return because the place order API does not return a JSONserializable response on success
# BUT the order will still go through.  Checked on the site.  You can also check your active orders below.

try:
    response = td_client.placeOrder(account_id, order.json())
    pprint(response)
except json.decoder.JSONDecodeError as err:
    print(err)


ValidationError: -1: An unknown error occurred

### Getting out Orders

In [12]:
# Load account info to get account ID for get orders
#td_client.accounts()

#account_id = td_client.accountIds[0]

In [13]:
# Print out orders
my_orders = td_client.orders(accountId=account_id)
pprint(my_orders)

[]


### Deleting Orders

In [19]:
# This deletes all WORKING orders

# for order in my_orders:
#     if order["status"] == Status.WORKING and order["cancelable"]:
#         response = td_client.cancelOrder(account_id, order["orderId"])
#         pprint(response)